# <font color='#fff200'> **Notebook Purpose and Introduction**

### **Purpose**
To test the rigor of the three previously defined criteria.

### **Introduction**
By including pieces of music with random notes, I establish a baseline for comparison. These pieces lack intentional tonality, coherent meter, and meaningful melodic structure. Contrasting them with music that adheres to these criteria helps highlight the importance of intentional tonality, meter, and melodic structure in creating meaningful music.

# **Necessary Libraries and Installations**

In [ ]:
!pip install mido

In [ ]:
!pip install midi2audio

In [ ]:
!pip install pydub

In [ ]:
!sudo apt-get install fluidsynth

In [ ]:
import librosa
import librosa.display
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import sys
import music21
import mido
import soundfile as sf
import subprocess

from librosa import midi_to_hz
from midi2audio import FluidSynth
from mido import MidiFile, MidiTrack, tempo2bpm
from pydub import AudioSegment
from music21 import converter, stream, meter
from fractions import Fraction
from scipy.signal import find_peaks

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

# **Random Music Generation**

In [ ]:
%cd /content/drive/Shareddrives/neotyagi-dataset
!git clone https://github.com/kevinzhang96/random-notes.git

In [ ]:
# Best Version Usage: !python filepath # iterations

# =============================================================================

# Original Repository: https://github.com/kevinzhang96/random-notes
# Edited By: Neo Tyagi
# Date: December 18th, 2023

# =============================================================================

# Original Author's Note:

# lots of code used from other sites; not all my own work!
# Some code from http://stackoverflow.com/a/311634/2605023 and
# http://soledadpenades.com/2009/10/29/fastest-way-to-generate-wav-files-in-python-using-the-wave-module/

# =============================================================================

import sys, os
import random
import struct, re
import numpy
import wave
import matplotlib.pyplot as plt

FPS = 44100                                                                     # the original number of frames per second in a wave file

WAVSECONDS = int(sys.argv[1])                                                   # the number of seconds in the wav
WAVLENGTH  = WAVSECONDS * FPS                                                   # the number of frames in the wav

CURRENTDIR = "/content/drive/Shareddrives/neotyagi-dataset/maestro-v3.0.0/MaestroTest/[5] Garbage Music Generation"                                                         # the current directory
WAVDIR     = CURRENTDIR                                                         # the wav file directory
DIRFILES   = [filename for filename in os.listdir(WAVDIR) if os.path.isfile(WAVDIR + filename)] # filter for files only
WAVPATTERN = re.compile('^randomwav\d{3}.wav')
WAVFILES   = [file for file in DIRFILES if WAVPATTERN.match(file)]              # filter for wav files formatted as 'randomwav###.wav'
WAVFILES.sort()                                                                 # sort by number; not necessary right now
WAVFILECOUNT = len(WAVFILES)                                                    # number of existing wavs

minFrequency = 100
maxFrequency = 6000

if WAVFILECOUNT > 1000:
  print ('Please clear your /wavs directory before continuing.')                # just for cleanliness
else:
    NewNumber = str(WAVFILECOUNT).zfill(3)                                      # next wav's index
    FileName  = WAVDIR + 'randomwav' + NewNumber + '.wav'                       # the wav's filename
    output = wave.open(FileName, 'w')                                           # create the wav
    output.setparams((2, 2, 44100, 0, 'NONE', 'not compressed'))                # set the wav file's properties

    value_str = b''                                                             # this will contain the frames we write

    if random.choice([True, False]):
      print("This piece is monophonic\n\n")
      for i in range(0, WAVSECONDS):
        baseNoteRate = random.uniform(2, 8)                                     # for each note to be played
        noteLengthFactor = random.uniform(0.6, 1.4)
        newNoteRate = baseNoteRate * noteLengthFactor
        value = random.choice(FREQARRAY)                                        # pick a random frequency
        print("Value chosen: ", value)

        if random.choice([True, False]):
          tempoChange = random.uniform(-0.5, 0.5)
          newNoteRate *= 1 + tempoChange

        print("newNoteRate: ", newNoteRate)
        period = float(FPS / newNoteRate) / float(value)                        # calculate the period
        omega = numpy.pi * 2 / period                                           # and the omega
        xaxis = numpy.arange(int(period), dtype = float) * omega                # get the x-value
        ydata = 16384 * numpy.sin(xaxis)                                        # create a sin-wave

        signal = numpy.resize(ydata, int(WAVLENGTH / baseNoteRate))             # resize the sin-wave
        # print("Signal Array: ", signal)

        ssignal = b''                                                           # temporary holding value

        for j in range(len(signal)):                                            # write the sin-wave to a string
          ssignal += struct.pack('h', int(signal[j]))
        value_str += ssignal                                                    # append the string to the total
    else:
      PCM_SCALE = 32767.0
      signal = numpy.zeros(int(WAVLENGTH))
      for i in range(0, WAVSECONDS):
        FREQARRAY = [random.uniform(minFrequency, maxFrequency) for _ in range(6000)]
        numNotesPerWAVSec = random.randint(2, 5)
        notesInWAVSecond = []
        maxNoteLength = 0

        for _ in range(numNotesPerWAVSec):
          baseNoteRate = random.uniform(2, 8)                                     # for each note to be played
          noteLengthRandomFactor = random.uniform(0.8, 1.2)
          newNoteRate = baseNoteRate * noteLengthRandomFactor
          if random.choice([True, False]):
            tempoChange = random.uniform(-0.6, 0.6)
            newNoteRate *= 1 + tempoChange
          if i == 0:
              note = {
              'frequency': random.choice(FREQARRAY) * random.uniform(0.90,1.35),
              'start': 0,
              'length': int(newNoteRate * FPS)
              }
          else:
            note = {
              'frequency': random.choice(FREQARRAY) * random.uniform(0.85,1.35),
              'start': maxNoteLength * random.uniform(0.15, 0.85),
              'length': int(newNoteRate * FPS)
            }
          notesInWAVSecond.append(note)
          maxNoteLength = max(maxNoteLength, note['length'])

        for note in notesInWAVSecond:
          NOTERATE = note['length'] / FPS
          period = float(FPS / NOTERATE) / float(note['frequency'])
          omega = numpy.pi * 2 / period
          xaxis = numpy.arange(int(period), dtype = float) * omega
          ydata = 16384 * numpy.sin(xaxis)

          startIndex = int(note['start'] + i * FPS)
          endIndex = int(startIndex + note['length'])
          print(f"Note Frequency: {note['frequency']} Hz")
          print(f"Start Index: {startIndex}")
          print(f"End Index: {endIndex}")
          print(f"Note Length: {note['length'] / FPS} seconds")
          print()

          ydata = numpy.resize(ydata, endIndex - startIndex)

          signal = numpy.resize(signal, max(endIndex, len(signal)))
          signal[slice(startIndex, endIndex)] += ydata                            # Accumulate the signal for each note

        signal /= numpy.max(numpy.abs(signal)) / PCM_SCALE
        print("\nFirst 10 Elements of Signal Array:", signal[:10])
        print("\nLast 10 Elements of Signal Array:", signal[-10:])
        print()
        print("=" * 100)
        print()
        ssignal = b''

        for j in range(len(signal)):
          ssignal += struct.pack('h', int(signal[j]))
        value_str += ssignal

    print()
    print("\nFirst Hundred Elements of Final Signal Array:", signal[:100])
    print("\nLast Hundred Elements of Final Signal Array:", signal[-100:])
    print()
    print('=' * 100)
    print()
    output.writeframes(value_str)                                               # write the frames to the wav
    output.close()                                                              # finished making wav

    if os.name == 'nt':                                                         # if on windows
        import winsound
        winsound.PlaySound(FileName, winsound.SND_FILENAME)                     # play using winsound
    elif os.name == 'posix':                                                    # else if on linux
        from pydub import AudioSegment
        from pydub.playback import play
        audio = AudioSegment.from_wav(FileName)
        play(audio)

    else:
        import ossaudiodev                                                      # else on UNIX
        s = wave.open(FileName, 'rb')                                           # play using ossaudio
        (nc, sw, fr, nf, comptype, compname) = s.getparams()
        dsp = ossaudiodev.open('/dev/dsp', 'w')
        try:
            from ossaudiodev import AFMT_S16_NE
        except ImportError:
            from sys import byteorder
            if byteorder == 'little':
                AFMT_S16_NE = ossaudiodev.AFMT_S16_LE
            else:
                AFMT_S16_NE = ossaudiodev.AFMT_S16_BE
        dsp.setparameters(AFMT_S16_NE, nc, fr)
        data = s.readframes(nf)
        s.close()
        dsp.write(data)
        dsp.close()

In [ ]:
!python /content/drive/Shareddrives/neotyagi-dataset/random-notes/generatemono.py 15

In [ ]:
!python /content/drive/Shareddrives/neotyagi-dataset/random-notes/generatepoly.py 5

# **Criteria 1 - Tonality**

**Running Pitch Detection on Each Piece**

In [ ]:
monophonicDirectory = "/content/drive/Shareddrives/neotyagi-dataset/maestro-v3.0.0/MaestroTest/"
noteInTuneTolerance = 7
pieceInTuneThreshold = 85

noteFrequencies = [
    65.41, 69.30, 73.42, 77.78, 82.41, 87.31, 92.50, 98.00, 103.83, 110.00, 116.54,
    123.47, 130.81, 138.59, 146.83, 155.56, 164.81, 174.61, 185.00, 196.00, 207.65,
    220.00, 233.08, 246.94, 261.63, 277.18, 293.66, 311.13, 329.63, 349.23, 369.99,
    392.00, 415.30, 440.00, 466.16, 493.88, 523.25, 554.37, 587.33, 622.25, 659.26,
    698.46, 739.99, 783.99, 830.61, 880.00, 932.33, 987.77, 1046.50, 1108.73, 1174.66,
    1244.51, 1318.51, 1396.91, 1479.98, 1567.98, 1661.22, 1760.00, 1864.66, 1975.53
]

def findClosestNoteFrequency(frequency):
    return min(noteFrequencies, key = lambda x: abs(x - frequency))

for filename in os.listdir(monophonicDirectory):
    totalNotes = 0
    if filename.endswith(".wav"):
        filepath = os.path.join(monophonicDirectory, filename)
        # filepathWAV = filepath[:-4] + '.wav'
        # fs = FluidSynth()
        # fs.midi_to_audio(filepath, filepath)
        audioData, sampleRate = librosa.load(filepath, sr=None)
        inTuneCount = 0

        frequencyEstimatesPYIN = librosa.pyin(audioData, fmin = librosa.note_to_hz('C2'), fmax = librosa.note_to_hz('C7'), fill_na = -1)
        frequencies = frequencyEstimatesPYIN[0]
        magnitudes = frequencyEstimatesPYIN[1]

        for i in range(0, len(frequencies), 5):
          frequency = frequencies[i]
          if frequency > 0:
            # print(f"This is the Frequency by PYIN: {frequency:.2f} Hz")
            closestNoteFrequency = findClosestNoteFrequency(frequency)
            # print(f"This is the closestNoteFrequency: {closestNoteFrequency:.2f} Hz")

            if abs(frequency - closestNoteFrequency) <= noteInTuneTolerance:
                inTuneCount += 1
            totalNotes+=1

        percentageInTune = (inTuneCount / totalNotes) * 100
        isPieceInTune = percentageInTune >= pieceInTuneThreshold

        print("")
        print(f"File: {filename}")
        print(f"Percentage of in-tune notes: {percentageInTune}%")
        print(f"The piece is{' ' if isPieceInTune else ' not '}in tune.\n")

# **Criteria 2 - Meter**

**Running Beat Detection on Each Piece**

In [ ]:
testDirectoryMeter = "/content/drive/Shareddrives/neotyagi-dataset/maestro-v3.0.0/MaestroTest/"

# This function checks if a piece has randomly distributed notes.
def hasRandomNotes(audioFile, maxStabilityThreshold = 0.07, maxNoteRepetitions = 5, maxNoteDistributionStability = 14):
    audioData, sampleRate = librosa.load(audioFile)

    cqt = librosa.cqt(y = audioData, sr = sampleRate) # Calculate the constant-Q chromagram [a representation of an audio signal that captures the energy distribution of different pitches over time].
    chromagram = librosa.amplitude_to_db(np.abs(cqt), ref=np.max) # Convert to decibels for easier analysis.

    # Calculating the note distribution stability
    noteDistributionStability = np.std(chromagram, axis=0).mean()

    onsetStrength = librosa.onset.onset_strength(y = audioData, sr = sampleRate) # Onset Strength: The magnitude of sudden changes in the audio signal [beginning of a note].
    tempo, _ = librosa.beat.beat_track(onset_envelope = onsetStrength, sr = sampleRate)
    _, beatTimes = librosa.beat.beat_track(onset_envelope = onsetStrength, sr = sampleRate, units = 'time') # Estimating the tempo and the beat times.

    meanTempoInterval = np.mean(np.diff(beatTimes))
    meanTempoIntervalStandardDeviation = np.std(np.diff(beatTimes))
    rhythmicStability = meanTempoIntervalStandardDeviation / meanTempoInterval

    noteOnsets = librosa.onset.onset_detect(y = audioData, sr = sampleRate) # Detects the beginning of notes and calculating the maximum note repetitions in the audio.
    noteIntervals = np.diff(noteOnsets)
    maxNoteReps = np.max(np.bincount(noteIntervals))

    if rhythmicStability >= maxStabilityThreshold or noteDistributionStability >= maxNoteDistributionStability or maxNoteReps >= maxNoteRepitions:
        return True, rhythmicStability, noteDistributionStability, maxNoteReps
    else:
        return False, rhythmicStability, noteDistributionStability, maxNoteReps

for filename in os.listdir(testDirectoryMeter):
    if filename.endswith(".wav"):
        filepath = os.path.join(monophonicDirectory, filename)
        # filepathWAV = filepath[:-4] + '.wav'
        # fs = FluidSynth()
        # fs.midi_to_audio(filepath, filepath)
        hasRandom, rhythmStability, noteDistributionStability, maxNoteRepitions = hasRandomNotes(filepath)
        if hasRandom:
            print(f"{filename}: The piece has randomly distributed notes.")
        else:
            print(f"{filename}: The piece has a good rhythm with structured note distribution.")
        print(f"Rhythmic Stability: {rhythmStability:.2f}")                     # A lower value of rhythmic stability indicates more stable rhythm patterns | higher = less stable rhythm patterns.
        print(f"Note Distribution Stability: {noteDistributionStability:.2f}")  # A lower value of note distribution stability indicates even distribution across the pitch range | higher = less even distribution
        print(f"Maximum Note Repititons: {maxNoteRepitions:.2f}")

# **Criteria 3 - Melodic Structure**

**Running Melodic Structure Detection on Each Piece**

In [ ]:
testDirectoryMelody = "/content/drive/Shareddrives/neotyagi-dataset/maestro-v3.0.0/MaestroTest/[6] Melodic Structure Test"

# Function to analyze melodic structure
def analyzeMelodicStructure(audioFile):
    audioData, samplingRate = librosa.load(audioFile)

    pitches, magnitudes = librosa.core.piptrack(y = audioData, sr = samplingRate) # Compute the pitch (fundamental frequency - lowest frequency) over time

    pitchContour = pitches[np.argmax(magnitudes, axis=0)]
    pitchContour = pitchContour.ravel()  # Flatten to a 1D Array.

    melodicIntervals = np.diff(pitchContour)                                    # Melodic intervals are the differences between consecutive pitch values.
    melodicPeaks, _ = find_peaks(pitchContour)
    melodicComplexity = np.std(melodicIntervals)                                # Measures the standard deviation of the intervals between consecutive pitches in the melodic contour.


    melodicStabilityRange = np.max(pitchContour) - np.min(pitchContour)         # Measures the span between the highest and lowest pitches in the melodic contour.
    noteDurations = np.diff(melodicPeaks) / samplingRate
    dynamicVariation = np.std(audioData)

    return melodicPeaks, melodicComplexity, melodicStabilityRange, noteDurations, dynamicVariation

for filename in os.listdir(testDirectoryMelody):
    if filename.endswith(".wav"):
        filepath = os.path.join(testDirectoryMelody, filename)
        melodicPeaks, melodicComplexity, melodicStabilityRange, noteDurations, dynamicVariation = analyzeMelodicStructure(filepath)  # Analyze melodic structure

        print(f'File: {filename}\n')

        print(f'Melodic Peaks: {melodicPeaks}')                                 # Peaks in the pitch contour can indicate potential melodic peaks or points of repetition.


        print(f'\nMelodic Complexity: {melodicComplexity}')                     # A higher melodic complexity indicates greater variation in the intervals between consecutive pitches.
                                                                                # This could suggest a more intricate and diverse melodic structure with complex patterns and variations.

                                                                                # A lower melodic complexity suggests more consistent intervals between pitches.
                                                                                # This could indicate a simpler or more repetitive melodic structure with less variation in pitch intervals.

        print(f'\nMelodic Stability: {melodicStabilityRange}')                  # A larger melodic stability range suggests a wider pitch range in the melody, indicating a more diverse or expansive melodic structure.
                                                                                # A smaller melodic stability range suggests a more focused or narrow pitch range, possibly indicating a simpler or repetitive melodic structure.

        print(f'\nAverage Note Duration: {np.mean(noteDurations):.4f} seconds') # A higher average note duration indicates that the notes in the melody are sustained for a longer time. This might suggest a slower-paced melodic style.
                                                                                # A lower average note duration suggests shorter note durations on average, which might indicate a faster-paced or more staccato melodic style.

        print(f'\nStandard Deviation of Note Duration: {np.std(noteDurations):.4f} seconds') # A higher standard deviation of note duration indicates greater variability in the duration of notes. This suggests a more dynamic melody.
                                                                                # A lower standard deviation suggests more consistent note durations, indicating a more regular and predictable rhythm.

        print(f'\nDynamic Variation: {dynamicVariation}')                       # A higher dynamic variation suggests a wider range of loudness levels in the melody. This might indicate a more expressive and dynamically varied performance.
                                                                                # A lower dynamic variation suggests a more consistent loudness level, which might indicate a more controlled or uniform performance.
        print()
        print('=' * 50)
        print()
